# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f49fdc3a580>
├── 'a' --> tensor([[-0.8848,  0.5760, -0.4321],
│                   [ 0.0136,  1.0800, -0.9075]])
└── 'x' --> <FastTreeValue 0x7f49fdc3adc0>
    └── 'c' --> tensor([[ 0.6837, -0.5424, -0.1344, -0.0606],
                        [ 1.0536,  0.3099,  0.4940,  0.7020],
                        [-0.7743,  0.8106,  0.5798,  0.5455]])

In [4]:
t.a

tensor([[-0.8848,  0.5760, -0.4321],
        [ 0.0136,  1.0800, -0.9075]])

In [5]:
%timeit t.a

74.2 ns ± 0.0709 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f49fdc3a580>
├── 'a' --> tensor([[ 1.3276,  0.2370, -2.5357],
│                   [-0.0716, -0.8963, -1.1157]])
└── 'x' --> <FastTreeValue 0x7f49fdc3adc0>
    └── 'c' --> tensor([[ 0.6837, -0.5424, -0.1344, -0.0606],
                        [ 1.0536,  0.3099,  0.4940,  0.7020],
                        [-0.7743,  0.8106,  0.5798,  0.5455]])

In [7]:
%timeit t.a = new_value

75.7 ns ± 0.0643 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.8848,  0.5760, -0.4321],
               [ 0.0136,  1.0800, -0.9075]]),
    x: Batch(
           c: tensor([[ 0.6837, -0.5424, -0.1344, -0.0606],
                      [ 1.0536,  0.3099,  0.4940,  0.7020],
                      [-0.7743,  0.8106,  0.5798,  0.5455]]),
       ),
)

In [10]:
b.a

tensor([[-0.8848,  0.5760, -0.4321],
        [ 0.0136,  1.0800, -0.9075]])

In [11]:
%timeit b.a

64.7 ns ± 0.172 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-2.7629, -0.7736, -0.3347],
               [-0.2208, -0.2554,  0.3909]]),
    x: Batch(
           c: tensor([[ 0.6837, -0.5424, -0.1344, -0.0606],
                      [ 1.0536,  0.3099,  0.4940,  0.7020],
                      [-0.7743,  0.8106,  0.5798,  0.5455]]),
       ),
)

In [13]:
%timeit b.a = new_value

542 ns ± 0.313 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

903 ns ± 3.47 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 32 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 1.86 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 739 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f492ddb3e80>
├── 'a' --> tensor([[[-0.8848,  0.5760, -0.4321],
│                    [ 0.0136,  1.0800, -0.9075]],
│           
│                   [[-0.8848,  0.5760, -0.4321],
│                    [ 0.0136,  1.0800, -0.9075]],
│           
│                   [[-0.8848,  0.5760, -0.4321],
│                    [ 0.0136,  1.0800, -0.9075]],
│           
│                   [[-0.8848,  0.5760, -0.4321],
│                    [ 0.0136,  1.0800, -0.9075]],
│           
│                   [[-0.8848,  0.5760, -0.4321],
│                    [ 0.0136,  1.0800, -0.9075]],
│           
│                   [[-0.8848,  0.5760, -0.4321],
│                    [ 0.0136,  1.0800, -0.9075]],
│           
│                   [[-0.8848,  0.5760, -0.4321],
│                    [ 0.0136,  1.0800, -0.9075]],
│           
│                   [[-0.8848,  0.5760, -0.4321],
│                    [ 0.0136,  1.0800, -0.9075]]])
└── 'x' --> <FastTreeValue 0x7f49f8ba6fd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.3 µs ± 40.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f49fdc52b80>
├── 'a' --> tensor([[-0.8848,  0.5760, -0.4321],
│                   [ 0.0136,  1.0800, -0.9075],
│                   [-0.8848,  0.5760, -0.4321],
│                   [ 0.0136,  1.0800, -0.9075],
│                   [-0.8848,  0.5760, -0.4321],
│                   [ 0.0136,  1.0800, -0.9075],
│                   [-0.8848,  0.5760, -0.4321],
│                   [ 0.0136,  1.0800, -0.9075],
│                   [-0.8848,  0.5760, -0.4321],
│                   [ 0.0136,  1.0800, -0.9075],
│                   [-0.8848,  0.5760, -0.4321],
│                   [ 0.0136,  1.0800, -0.9075],
│                   [-0.8848,  0.5760, -0.4321],
│                   [ 0.0136,  1.0800, -0.9075],
│                   [-0.8848,  0.5760, -0.4321],
│                   [ 0.0136,  1.0800, -0.9075]])
└── 'x' --> <FastTreeValue 0x7f4935f41c70>
    └── 'c' --> tensor([[ 0.6837, -0.5424, -0.1344, -0.0606],
                        [ 1.0536,  0.3099,  0.4940,  0.7020],
                 

In [23]:
%timeit t_cat(trees)

31 µs ± 34.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

67.6 µs ± 111 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.8848,  0.5760, -0.4321],
                [ 0.0136,  1.0800, -0.9075]],
       
               [[-0.8848,  0.5760, -0.4321],
                [ 0.0136,  1.0800, -0.9075]],
       
               [[-0.8848,  0.5760, -0.4321],
                [ 0.0136,  1.0800, -0.9075]],
       
               [[-0.8848,  0.5760, -0.4321],
                [ 0.0136,  1.0800, -0.9075]],
       
               [[-0.8848,  0.5760, -0.4321],
                [ 0.0136,  1.0800, -0.9075]],
       
               [[-0.8848,  0.5760, -0.4321],
                [ 0.0136,  1.0800, -0.9075]],
       
               [[-0.8848,  0.5760, -0.4321],
                [ 0.0136,  1.0800, -0.9075]],
       
               [[-0.8848,  0.5760, -0.4321],
                [ 0.0136,  1.0800, -0.9075]]]),
    x: Batch(
           c: tensor([[[ 0.6837, -0.5424, -0.1344, -0.0606],
                       [ 1.0536,  0.3099,  0.4940,  0.7020],
                       [-0.7743,  0.8106,  0.5798,  0.5455]],
         

In [26]:
%timeit Batch.stack(batches)

84 µs ± 73.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.8848,  0.5760, -0.4321],
               [ 0.0136,  1.0800, -0.9075],
               [-0.8848,  0.5760, -0.4321],
               [ 0.0136,  1.0800, -0.9075],
               [-0.8848,  0.5760, -0.4321],
               [ 0.0136,  1.0800, -0.9075],
               [-0.8848,  0.5760, -0.4321],
               [ 0.0136,  1.0800, -0.9075],
               [-0.8848,  0.5760, -0.4321],
               [ 0.0136,  1.0800, -0.9075],
               [-0.8848,  0.5760, -0.4321],
               [ 0.0136,  1.0800, -0.9075],
               [-0.8848,  0.5760, -0.4321],
               [ 0.0136,  1.0800, -0.9075],
               [-0.8848,  0.5760, -0.4321],
               [ 0.0136,  1.0800, -0.9075]]),
    x: Batch(
           c: tensor([[ 0.6837, -0.5424, -0.1344, -0.0606],
                      [ 1.0536,  0.3099,  0.4940,  0.7020],
                      [-0.7743,  0.8106,  0.5798,  0.5455],
                      [ 0.6837, -0.5424, -0.1344, -0.0606],
                      [ 1.0536,  

In [28]:
%timeit Batch.cat(batches)

155 µs ± 320 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

352 µs ± 6.46 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
